## 1. Импорт пакетов и файла

In [2]:
%matplotlib inline
import pandas as pd
import string
import re
import nltk
import pymorphy3

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('games.csv').drop('Unnamed: 0', axis=1)

In [4]:
df.head()

,title,description,date,tags,rating
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10


## 2. Предварительная обработка данных

In [6]:
import re

def remove_english_words(text):
    # Удаляет слова, состоящие только из английских букв (включая сокращения)
    return re.sub(r'\b[a-zA-Z]+\b', '', text)

def remove_punctuation(text): 
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text): 
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text): 
    return re.sub(r'\s+', ' ', text, flags=re.I)

st = '❯\xa0—«»'
def remove_othersymbol(text):
    return ''.join([ch if ch not in st else ' ' for ch in text])

In [7]:
df['prep_text'] = [remove_english_words(remove_multiple_spaces(remove_numbers(remove_othersymbol(remove_punctuation(text.lower()))))) for text in df['description']]

In [8]:
df.head()

,title,description,date,tags,rating,prep_text
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10,более двух десятилетий служит примером перво...
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10,раз в год художница просыпается и рисует на мо...
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10,последняя линия нападения галактики станьте ад...
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10,приключенческая ролевая игра с открытым миром...
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10,самая популярная игра в steamежедневно миллион...


In [9]:
df['description'][0]

'Более двух десятилетий Counter-Strike служит примером первоклассной соревновательной игры, путь развития которой определяют миллионы игроков со всего мира. Теперь пришло время нового этапа — Counter-Strike 2.Counter-Strike 2 — это бесплатное улучшение для CS:GO, которое знаменует собой крупнейший технологический скачок в истории серии. Оно разработано на движке Source 2 и модернизирует игру благодаря реалистичному и физически корректному рендерингу, организации сети по последнему слову технологий и улучшенным инструментам для мастерской сообщества.Кроме представленного в 1999 году классического игрового процесса, завязанного на достижении цели, Counter-Strike 2 включает:новые рейтинги CS и обновлённый премьер-режим;глобальные и региональные таблицы лидеров;улучшенные и воссозданные с нуля карты;принципиально новый динамический дым от гранат;игровой процесс, не зависящий от тикрейта;переосмысленные визуальные эффекты и звуки;все предметы инвентаря из CS:GO.'

In [10]:
df['prep_text'][0]

'более двух десятилетий   служит примером первоклассной соревновательной игры путь развития которой определяют миллионы игроков со всего мира теперь пришло время нового этапа     это бесплатное улучшение для   которое знаменует собой крупнейший технологический скачок в истории серии оно разработано на движке  и модернизирует игру благодаря реалистичному и физически корректному рендерингу организации сети по последнему слову технологий и улучшенным инструментам для мастерской сообщества кроме представленного в году классического игрового процесса завязанного на достижении цели   включает новые рейтинги  и обновлённый премьер режим глобальные и региональные таблицы лидеров улучшенные и воссозданные с нуля карты принципиально новый динамический дым от гранат игровой процесс не зависящий от тикрейта переосмысленные визуальные эффекты и звуки все предметы инвентаря из   '

In [11]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['издание', 'игра', 'игре', 'т.д.', 'т', 'д', 'это','который','с','своём','всем','наш', 'свой', 'об этой игре', 'это', 'в', 'на', 'и', 'или', 'для', 'что', 'как', 'также', 'с', 'от', 'до','по', 'из', 'при', 'во', 'без', 'о', 'об', 'у', 'а', 'но', 'же', 'ли', 'бы','быть', 'был', 'была', 'будет', 'есть', 'нет', 'все', 'его', 'ее', 'их','мы', 'вы', 'они', 'наш', 'ваш', 'который', 'этот', 'такие', 'тоже','игра', 'игры', 'игровой', 'игрок', 'игроки', 'режим', 'режима', 'режимы','поддержка', 'поддерживает', 'доступен', 'доступна', 'доступны', 'поддерживает','уровень', 'уровни', 'миссия', 'миссии', 'задания', 'задание', 'команда','команды', 'персонаж', 'персонажи', 'система', 'системы', 'возможность','возможности', 'новый', 'новая', 'новые', 'мир', 'мира', 'миры', 'разные','различные', 'вид', 'виды', 'тип', 'типы', 'версия', 'версии', 'контент','контента', 'обновление', 'обновления', 'дополнение', 'дополнения','доступ', 'платформа', 'платформы', 'windows', 'steam', 'pc', 'ps4', 'ps5', 'xbox','онлайн', 'singleplayer', 'multiplayer', 'coop', 'игровой', 'механика', 'жанр','можно', 'нужно', 'будет', 'более', 'менее', 'позволяет', 'таким', 'образом', 'далее', 'после', 'кроме', 'ещё', 'включая', 'включает', 'основе', 'основе','впереди', 'перед', 'через', 'время', 'только', 'особый', 'каждый', 'несколько', 'разный', 'возможный', 'невероятный','уникальный', 'главный', 'разнообразие', 'бесконечный', 'необычный',    'множество', 'много', 'достаточно', 'большой', 'огромный', 'маленький',    'весь', 'самый', 'новый', 'поздний', 'ранний', 'вторжение', 'второй',    'последний', 'следующий', 'будущий', 'собственный', 'свой', 'ваш', 'наш','поэтому', 'почему', 'только', 'очень', 'поистине', 'просто', 'вполне',    'возможно', 'наконец', 'всего', 'почти', 'даже', 'лишь',    'путешествие', 'отправиться', 'отправляться', 'поездка', 'поезд',    'охота', 'охотник', 'персонажа', 'персонажейв', 'персонажава',    'экспедиция', 'навык', 'мастерство', 'исследовать', 'изучать',    'отряд', 'миссия', 'задание', 'сражение', 'поединок', 'план', 'поиск',    'приключение', 'битва', 'бой', 'поединок', 'путь', 'борьба','магия', 'чудовище', 'призрак', 'город', 'герой', 'враг', 'союзник',    'история', 'сюжет', 'роль', 'предмет', 'объект', 'вещь', 'мир',    'вселенная', 'пространство', 'время', 'реальность', 'облик', 'формировать',    'режим', 'игра', 'игрок', 'миссия', 'интерфейс', 'настройка', 'поезд',    'уровень', 'система', 'режимосновать', 'персонаж', 'предмет',    'обновление', 'поддержка', 'боевой', 'арена', 'кампания', 'редакция',    'особняк', 'станция', 'платформенный', 'платформа', 'персонализация',    'выживание', 'снаряжение', 'возможность', 'объект', 'знание', 'истина',    'настроить', 'подстроить', 'использовать', 'предлагать', 'раскрыть',    'создавать', 'создаваться', 'включать', 'выйти', 'процесс','находить', 'получать', 'узнать', 'освоить', 'прийтись', 'пройти','стараться', 'приходиться', 'представить', 'превзойти', 'позволять','предсказывать', 'адаптироваться', 'создавать', 'разрабатывать','выполнять', 'происходить', 'отражать', 'направить', 'проникать','распоряжение', 'двигаться', 'возникать', 'вмешиваться', 'жить','открывать', 'закрывать', 'ожидать', 'включать', 'перейти', 'позволить']) 
russian_stopwords 

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [12]:
def tokenize(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    text = " ".join(tokens)
    return text

In [13]:
df['tokenize_text'] = [tokenize(text) for text in df['prep_text']]

In [14]:
df.head()

,title,description,date,tags,rating,prep_text,tokenize_text
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10,более двух десятилетий служит примером перво...,двух десятилетий служит примером первоклассной...
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10,раз в год художница просыпается и рисует на мо...,год художница просыпается рисует монолите выво...
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10,последняя линия нападения галактики станьте ад...,последняя линия нападения галактики станьте ад...
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10,приключенческая ролевая игра с открытым миром...,приключенческая ролевая открытым миром рассказ...
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10,самая популярная игра в steamежедневно миллион...,самая популярная steamежедневно миллионы игрок...


In [15]:
stemmer = SnowballStemmer("russian")

stem_list = []
for text in (df['tokenize_text']):
    try:
        tokens = word_tokenize(text)
        res = list()
        for word in tokens:
            p = stemmer.stem(word)
            res.append(p)
        text = " ".join(res)
        stem_list.append(text)
    except Exception as e:
        print(e)
        
df['text_stem'] = stem_list

In [16]:
df.head()

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10,более двух десятилетий служит примером перво...,двух десятилетий служит примером первоклассной...,двух десятилет служ пример первоклассн соревно...
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10,раз в год художница просыпается и рисует на мо...,год художница просыпается рисует монолите выво...,год художниц просыпа рис монол вывод сво прокл...
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10,последняя линия нападения галактики станьте ад...,последняя линия нападения галактики станьте ад...,последн лин нападен галактик станьт адск десан...
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10,приключенческая ролевая игра с открытым миром...,приключенческая ролевая открытым миром рассказ...,приключенческ ролев открыт мир рассказыва кибе...
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10,самая популярная игра в steamежедневно миллион...,самая популярная steamежедневно миллионы игрок...,сам популярн стеамежедневн миллион игрок всем ...


In [17]:
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [18]:
%%time
lemm_texts_list = []
for text in (df['tokenize_text']):
    try:
        tokens = word_tokenize(text)
        res = list()
        for word in tokens:
            p = morph.parse(word)[0]
            res.append(p.normal_form)
        text = " ".join(res)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df['text_lemm'] = lemm_texts_list

CPU times: total: 14.1 s
Wall time: 14.9 s


In [19]:
df.head()

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10,более двух десятилетий служит примером перво...,двух десятилетий служит примером первоклассной...,двух десятилет служ пример первоклассн соревно...,два десятилетие служить пример первоклассный с...
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10,раз в год художница просыпается и рисует на мо...,год художница просыпается рисует монолите выво...,год художниц просыпа рис монол вывод сво прокл...,год художница просыпаться рисовать монолит выв...
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10,последняя линия нападения галактики станьте ад...,последняя линия нападения галактики станьте ад...,последн лин нападен галактик станьт адск десан...,последний линия нападение галактика стать адск...
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10,приключенческая ролевая игра с открытым миром...,приключенческая ролевая открытым миром рассказ...,приключенческ ролев открыт мир рассказыва кибе...,приключенческий ролевый открытый мир рассказыв...
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10,самая популярная игра в steamежедневно миллион...,самая популярная steamежедневно миллионы игрок...,сам популярн стеамежедневн миллион игрок всем ...,самый популярный steamежедневно миллион игрок ...


In [20]:
df['text_lemm'] = [tokenize(text) for text in df['text_lemm']]

## Векторизация

In [22]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=100000000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))

In [23]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_lemm'])

In [24]:
tfidf_matrix.shape

(500, 3070)

## Тематическое моделирование

In [26]:
from sklearn.decomposition import TruncatedSVD

In [27]:
# создание модели LSA
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

Topic 1: друг, оружие, мочь, создать, карта, играть, стать, сила, ждать, помощь
Topic 2: автомобиль, гонка, описание, трасса, гоночный, машина, чемпионат, вождение, реалистичный, физика
Topic 3: описание, ресурс, остров, выжить, планета, оружие, строить, земля, древний, собирать
Topic 4: остров, ресурс, строить, автомобиль, корабль, построить, дом, жизнь, животное, ферма
Topic 5: корабль, планета, война, галактика, космический, фракция, империя, техника, ресурс, космос


In [28]:
from sklearn.decomposition import NMF

In [29]:
# создание модели NMF
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

Topic 1: оружие, карта, друг, стиль, стать, класс, ждать, способность, сражаться, играть
Topic 2: автомобиль, гонка, гоночный, трасса, машина, реалистичный, физика, вождение, чемпионат, реальный
Topic 3: описание, открытый, поколение, матч, зона, впечатление, группировка, ребёнок, развивать, убежище
Topic 4: остров, ресурс, строить, жизнь, друг, дом, мочь, выжить, построить, инструмент
Topic 5: корабль, война, планета, галактика, империя, фракция, космический, стратегия, армия, звёздный


## Кластеризация

In [31]:
num_clusters = 5

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters, random_state=0)

In [32]:
km.fit(tfidf_matrix)

  File "C:\Users\artembruh\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\artembruh\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\artembruh\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\artembruh\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


KMeans(n_clusters=5, random_state=0)

In [33]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [34]:
clusterkm = km.labels_.tolist()
df['cluster'] = clusterkm

In [35]:
df.head(10)

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10,более двух десятилетий служит примером перво...,двух десятилетий служит примером первоклассной...,двух десятилет служ пример первоклассн соревно...,десятилетие служить пример первоклассный сорев...,1
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10,раз в год художница просыпается и рисует на мо...,год художница просыпается рисует монолите выво...,год художниц просыпа рис монол вывод сво прокл...,год художница просыпаться рисовать монолит выв...,1
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10,последняя линия нападения галактики станьте ад...,последняя линия нападения галактики станьте ад...,последн лин нападен галактик станьт адск десан...,линия нападение галактика стать адский десантн...,2
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10,приключенческая ролевая игра с открытым миром...,приключенческая ролевая открытым миром рассказ...,приключенческ ролев открыт мир рассказыва кибе...,приключенческий ролевый открытый рассказывать ...,4
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10,самая популярная игра в steamежедневно миллион...,самая популярная steamежедневно миллионы игрок...,сам популярн стеамежедневн миллион игрок всем ...,популярный steamежедневно миллион сражаться ли...,1
5,RoadCraft,"Вы – субподрядчик компании, которая специализи...","20 May, 2025","Симулятор, Строительство, Вождение, Физика, Пр...",7 / 10,вы – субподрядчик компании которая специализир...,– субподрядчик компании которая специализирует...,– субподрядчик компан котор специализир восста...,– субподрядчик компания специализироваться вос...,0
6,Baldur's Gate 3,Соберите отряд и вернитесь в Забытые Королевст...,"3 Aug, 2023","Ролевая игра, Кастомизация персонажа, Решения ...",10 / 10,соберите отряд и вернитесь в забытые королевст...,соберите вернитесь забытые королевства ждет др...,собер верн забыт королевств ждет дружб предате...,собрать вернуться забытый королевство ждать др...,4
7,DOOM: The Dark Ages,СТАНЬТЕ ПАЛАЧОМ В СРЕДНЕВЕКОВОЙ ВОЙНЕ С АДОМDO...,"14 May, 2025","Экшен, Шутер от первого лица, Демоны, Тёмное ф...",9 / 10,станьте палачом в средневековой войне с адомdo...,станьте палачом средневековой войне адомdoom с...,станьт палач средневеков войн адомдо студ зрел...,стать палач средневековый война адомdoom студи...,4
8,Tainted Grail: The Fall of Avalon,Добро пожаловать в мир «Tainted Grail: The Fal...,"23 May, 2025","Приключение, Открытый мир, Ролевая игра, Тёмно...",9 / 10,добро пожаловать в мир шагните в мрачную...,добро пожаловать шагните мрачную вселенную рол...,добр пожалова шагн мрачн вселен ролев перв лиц...,добро пожаловать шагнуть мрачный ролевый первы...,3
9,The Elder Scrolls IV: Oblivion Remastered,The Elder Scrolls IV: Oblivion™ Remastered пре...,"22 Apr, 2025","Ролевая игра, Открытый мир, Фэнтези, Для одног...",9 / 10,™ представляет собой переосмысление лучше...,™ представляет собой переосмысление лучшей год...,™ представля соб переосмыслен лучш год потряса...,™ представлять переосмысление хороший год потр...,3


In [36]:
df['cluster'].value_counts()

cluster
1    213
4    119
3     61
2     59
0     48
Name: count, dtype: int64

In [37]:
df[df['cluster']==0]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
5,RoadCraft,"Вы – субподрядчик компании, которая специализи...","20 May, 2025","Симулятор, Строительство, Вождение, Физика, Пр...",7 / 10,вы – субподрядчик компании которая специализир...,– субподрядчик компании которая специализирует...,– субподрядчик компан котор специализир восста...,– субподрядчик компания специализироваться вос...,0
20,Drive Beyond Horizons,Отправляйтесь в эпическое путешествие за прикл...,"24 Mar, 2025","Ранний доступ, Приключение, Автосимулятор, Экш...",9 / 10,отправляйтесь в эпическое путешествие за прикл...,отправляйтесь эпическое приключениями исследов...,отправля эпическ приключен исследован проедьт ...,эпический исследование проедить автомобиль обш...,0
38,Euro Truck Simulator 2,Станьте королем европейских дорог — водителем ...,"12 Oct, 2012","Вождение, Транспорт, Симулятор, Открытый мир, ...",10 / 10,станьте королем европейских дорог водителем гр...,станьте королем европейских дорог водителем гр...,станьт корол европейск дорог водител грузовик ...,стать король европейский дорога водитель грузо...,0
41,Farming Simulator 25,Игра «Симулятор фермы 25» приглашает в приятну...,"12 Nov, 2024","Симулятор, Симулятор фермы, Реализм, 3D, От пе...",9 / 10,игра симулятор фермы приглашает в приятную сел...,симулятор фермы приглашает приятную сельскую ж...,симулятор ферм приглаша приятн сельск жизн ста...,симулятор ферма приглашать приятный сельский ж...,0
50,BeamNG.drive,BeamNG.drive — невероятно реалистичный автосим...,"29 May, 2015","Симулятор, Вождение, Физика, Реализм, Разрушен...",10 / 10,невероятно реалистичный автосимулятор с прак...,невероятно реалистичный автосимулятор практиче...,невероятн реалистичн автосимулятор практическ ...,невероятно реалистичный автосимулятор практиче...,0
51,Forza Horizon 5,Вас ждёт бесконечный калейдоскоп приключений H...,"8 Nov, 2021","Гонки, Открытый мир, Вождение, Для нескольких ...",9 / 10,вас ждёт бесконечный калейдоскоп приключений ...,ждёт калейдоскоп приключений совершайте увлека...,ждет калейдоскоп приключен соверша увлекательн...,ждать калейдоскоп совершать увлекательный неве...,0
54,SnowRunner,Встречайте новое поколение гонок по бездорожью...,"17 May, 2021","Симулятор, Автосимулятор, Для нескольких игрок...",9 / 10,встречайте новое поколение гонок по бездорожью...,встречайте новое поколение гонок бездорожью ся...,встреча нов поколен гонок бездорож сядьт рул к...,встречать поколение гонка бездорожье сесть рул...,0
66,The Precinct,Вы офицер по имени Ник Корделл-младший — вчера...,"13 May, 2025","Детектив, Криминал, Песочница, Открытый мир, Ш...",9 / 10,вы офицер по имени ник корделл младший вчерашн...,офицер имени ник корделл младший вчерашний вып...,офицер имен ник корделл младш вчерашн выпускни...,офицер имя ник корделла младший вчерашний выпу...,0
88,American Truck Simulator,Ощутите мощь легендарных американских грузовик...,"2 Feb, 2016","Автосимулятор, Вождение, Открытый мир, Транспо...",10 / 10,ощутите мощь легендарных американских грузовик...,ощутите мощь легендарных американских грузовик...,ощут мощ легендарн американск грузовик доставл...,ощутить мощь легендарный американский грузовик...,0
89,JDM: Japanese Drift Master,Исследуйте родину дрифта и погрузитесь в культ...,"21 May, 2025","Гонки, Симулятор, Открытый мир, Автосимулятор,...",7 / 10,исследуйте родину дрифта и погрузитесь в культ...,исследуйте родину дрифта погрузитесь культовую...,исслед родин дрифт погруз культов автомобильн ...,родина дрифт погрузиться культовый автомобильн...,0


In [171]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=100000000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==0]['text_lemm'])
print('модель LSA:')
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель LSA:
Topic 1: автомобиль, гонка, гоночный, трасса, машина, реальный, мочь, помощь, создать, пилот
Topic 2: пилот, виртуальный, трасса, внутриигровой, гоночный, болид, гонка, впервые, приобретение виртуальный, виртуальный внутриигровой
Topic 3: ферма, внутриигровой, виртуальный, пилот, внутриигровой покупка, покупка виртуальный валюта, покупка виртуальный, приобретение, приобретение виртуальный, приобретение виртуальный внутриигровой
Topic 4: ферма, фермер, симулятор ферма, поле, сельскохозяйственный, фермерский, сельский, прочий, хозяйство, лесоводство
Topic 5: грузовик, груз, доставлять, гоночный, водитель, вариант, симуляция, вариант вариант, мотоцикл, ферма
модель NMF:
Topic 1: гоночный, гонка, автомобиль, трасса, реальный, трек, заезд, дрифт, чемпионат, включить
Topic 2: пилот, внутриигровой, виртуальный, приобретение виртуальный внутриигровой, виртуальный валюта, дополнительный внутриигровой, виртуальный валюта приобретение, приобретение виртуальный, приобретение, предусмот

In [39]:
df[df['cluster']==1]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
0,Counter-Strike 2,Более двух десятилетий Counter-Strike служит п...,"21 Aug, 2012","Шутер от первого лица, Шутер, Для нескольких и...",9 / 10,более двух десятилетий служит примером перво...,двух десятилетий служит примером первоклассной...,двух десятилет служ пример первоклассн соревно...,десятилетие служить пример первоклассный сорев...,1
1,Clair Obscur: Expedition 33,Раз в год Художница просыпается и рисует на Мо...,"24 Apr, 2025","Пошаговые сражения, Глубокий сюжет, Фэнтези, И...",10 / 10,раз в год художница просыпается и рисует на мо...,год художница просыпается рисует монолите выво...,год художниц просыпа рис монол вывод сво прокл...,год художница просыпаться рисовать монолит выв...,1
4,Dota 2,Самая популярная игра в SteamЕжедневно миллион...,"9 Jul, 2013","Бесплатная игра, MOBA, Для нескольких игроков,...",9 / 10,самая популярная игра в steamежедневно миллион...,самая популярная steamежедневно миллионы игрок...,сам популярн стеамежедневн миллион игрок всем ...,популярный steamежедневно миллион сражаться ли...,1
10,Schedule I,"Наркотики, вещества или химикаты Списка I опре...","24 Mar, 2025","Симулятор, Кооператив, Криминал, Для нескольки...",10 / 10,наркотики вещества или химикаты списка опреде...,наркотики вещества химикаты списка определяютс...,наркотик веществ химикат списк определя лекарс...,наркотик вещество химикат список определяться ...,1
11,R.E.P.O.,"R.E.P.O. это онлайн-игра в ужасе, в которой уч...","26 Feb, 2025","Хоррор, Сетевой кооператив, Для нескольких игр...",10 / 10,это онлайн игра в ужасе в которой участвую...,ужасе которой участвуют физика голосовой чат б...,ужас котор участв физик голосов чат близост ст...,ужас участвовать физика голосовой чат близость...,1
13,Warframe,Пробудитесь в роли неудержимого воина и сражай...,"25 Mar, 2013","Бесплатная игра, Ролевой экшен, Лутер-шутер, Ш...",9 / 10,пробудитесь в роли неудержимого воина и сражай...,пробудитесь роли неудержимого воина сражайтесь...,пробуд рол неудержим воин сража вмест друз сюж...,пробудиться неудержимый воин сражаться вместе ...,1
14,"Warhammer 40,000: Space Marine 2",Обретите сверхчеловеческую мощь космодесантник...,"9 Sep, 2024","Warhammer 40K, Экшен, Шутер от третьего лица, ...",9 / 10,обретите сверхчеловеческую мощь космодесантник...,обретите сверхчеловеческую мощь космодесантник...,обрет сверхчеловеческ мощ космодесантник пуст ...,обрести сверхчеловеческий мощь космодесантник ...,1
15,FANTASY LIFE i: The Girl Who Steals Time,"Погрузитесь в эту неторопливую RPG, где вы смо...","21 May, 2025","Ролевая игра, Экшен, Казуальная игра, Японская...",9 / 10,погрузитесь в эту неторопливую где вы сможете...,погрузитесь неторопливую сможете свободно пере...,погруз нетороплив сможет свободн переключа уни...,погрузиться неторопливый смочь свободно перекл...,1
16,Rust,Единственная цель в Rust — выживание. Дикая пр...,"8 Feb, 2018","Выживание, Крафтинг, Для нескольких игроков, О...",9 / 10,единственная цель в выживание дикая природа о...,единственная цель дикая природа острова обитат...,единствен цел дик природ остров обитател окруж...,единственный цель дикий природа остров обитате...,1
17,Apex Legends™,Побеждайте с характером в Apex Legends™ — бесп...,"4 Nov, 2020","Бесплатная игра, Для нескольких игроков, Корол...",6 / 10,побеждайте с характером в ™ бесплатном героич...,побеждайте характером ™ бесплатном героическом...,побежда характер ™ бесплатн героическ шутер ле...,побеждать характер ™ бесплатный героический шу...,1


In [173]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=100000000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==1]['text_lemm'])
print('модель LSA:')
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель LSA:
Topic 1: оружие, карта, мочь, друг, остров, создать, сражаться, играть, любой, ресурс
Topic 2: описание, матч, прогресс, открытый, вылазка, зона, соревнование, элемент, ощущение, модель
Topic 3: остров, ресурс, строить, построить, дом, жизнь, собирать ресурс, выжить, поселение, животное
Topic 4: техника, карта, война, остров, фракция, солдат, поле, товар, ресурс, тактический
Topic 5: карта, колода, зомби, карточный, коллекция, кки, остров, дуэль, реликвия, комбинация
модель NMF:
Topic 1: оружие, босс, класс, стиль, друг, способность, мочь, играть, умение, сражаться
Topic 2: описание, матч, прогресс, открытый, вылазка, соревнование, зона, ощущение, друг, экономика
Topic 3: остров, ресурс, строить, построить, дом, выжить, жизнь, поселение, собирать, инструмент
Topic 4: война, техника, фракция, поле, тактический, солдат, оружие, многопользовательский, армия, юнит
Topic 5: карта, колода, зомби, карточный, сетевой, коллекция, кки, комбинация, дуэль, сетевой карта


In [41]:
df[df['cluster']==2]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
2,HELLDIVERS™ 2,Последняя линия нападения галактики.Станьте Ад...,"8 Feb, 2024","Сетевой кооператив, Игрок против ИИ, Шутер от ...",7 / 10,последняя линия нападения галактики станьте ад...,последняя линия нападения галактики станьте ад...,последн лин нападен галактик станьт адск десан...,линия нападение галактика стать адский десантн...,2
27,Stellaris,Встречайте бесчисленное множество инопланетных...,"9 May, 2016","Космос, Глобальная стратегия, Стратегия, Научн...",9 / 10,встречайте бесчисленное множество инопланетных...,встречайте бесчисленное инопланетных рас взаим...,встреча бесчислен инопланетн рас взаимодейств ...,встречать бесчисленный инопланетный раса взаим...,2
65,The Planet Crafter,ОТ 1 ДО 10 ИГРОКОВНеторопливо отстраивайте сво...,"10 Apr, 2024",Симулятор выживания и крафтинга в открытом мир...,10 / 10,от до игроковнеторопливо отстраивайте свою пла...,игроковнеторопливо отстраивайте планету одиноч...,игроковнетороплив отстраива планет одиночк воз...,игроковнеторопливый отстраивать планета одиноч...,2
83,Phasmophobia,Phasmophobia-это 4-пенсерская онлайн-кооперати...,"18 Sep, 2020","Хоррор, Сетевой кооператив, Для нескольких игр...",10 / 10,это пенсерская онлайн кооператив психологичес...,пенсерская кооператив психологическая ужасов в...,пенсерск кооперат психологическ ужас ваш паран...,пенсерский кооператив психологический ужас пар...,2
100,Microsoft Flight Simulator 2024,Полет со смыслом Начните карьеру в авиации там...,"19 Nov, 2024","Симулятор, Полёты, Реализм, VR, Для нескольких...",6 / 10,полет со смыслом начните карьеру в авиации там...,полет смыслом начните карьеру авиации находите...,полет смысл начн карьер авиац наход набира опы...,полёт смысл начать карьера авиация находиться ...,2
105,STAR WARS™: The Old Republic™,STAR WARS™: The Old Republic™ — это единственн...,"21 Jul, 2020","Бесплатная игра, MMORPG, Для нескольких игроко...",9 / 10,™ ™ это единственная бесплатная многопользо...,™ ™ единственная бесплатная многопользовательс...,™ ™ единствен бесплатн многопользовательск кот...,™ ™ единственный бесплатный многопользовательс...,2
119,Satisfactory,Satisfactory — это игра от первого лица в откр...,"10 Sep, 2024","Строительство базы, Автоматизация, Открытый ми...",10 / 10,это игра от первого лица в открытом мире вы к...,первого лица открытом мире которой строить зав...,перв лиц открыт мир котор стро завод упор иссл...,первый лицо открытый строить завод упор исслед...,2
136,World of Warships,Скачайте World of Warships и играйте — абсолют...,"15 Nov, 2017","Бесплатная игра, Морской бой, Экшен, Кооперати...",7 / 10,скачайте и играйте абсолютно бесплатно вы м...,скачайте играйте абсолютно бесплатно можете пр...,скача игра абсолютн бесплатн может продвига бе...,скачать играть абсолютно бесплатно мочь продви...,2
163,"Warhammer 40,000: Gladius - Relics of War",Гладиус Прайм был известен среди Имперских уче...,"12 Jul, 2018","Стратегия, Warhammer 40K, 4X, Пошаговая страте...",9 / 10,гладиус прайм был известен среди имперских уче...,гладиус прайм известен среди имперских ученых ...,гладиус прайм извест сред имперск учен планет ...,гладиус прайма известный среди имперский учёны...,2
178,Cubic Odyssey,"Отправься в бескрайний мир, где ты можешь изме...","14 May, 2025",Симулятор выживания и крафтинга в открытом мир...,7 / 10,отправься в бескрайний мир где ты можешь измен...,отправься бескрайний можешь изменить каждую пл...,отправ бескрайн можеш измен кажд планет кажд с...,бескрайний мочь изменить планета миг полный жи...,2


In [175]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=100000000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==2]['text_lemm'])
print('модель LSA:')
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель LSA:
Topic 1: корабль, планета, галактика, космический, ресурс, мочь, друг, война, создать, звёздный
Topic 2: галактика, война, звёздный война, звёздный, галактический, корабль, империя, картель, джедай, сторона
Topic 3: предатель, член, экипаж, член экипаж, карта, задача, чат, корабль, заразить, друг
Topic 4: война, звёздный война, звёздный, джедай, существо, картель, пещера, странный, галактический, событие
Topic 5: член экипаж, завод, предатель, экипаж, член, заразить, задача, пыль, событие, ресурс
модель NMF:
Topic 1: корабль, планета, космический, база, ресурс, империя, фракция, экономика, флот, технология
Topic 2: галактика, война, звёздный война, звёздный, галактический, сторона, джедай, картель, планета, преступный
Topic 3: предатель, член экипаж, экипаж, член, задача, заразить, чат, убивать, саботаж, ограниченный событие
Topic 4: странный, существо, планета, изучить, создать, секрет, загадочный, строить, ресурс, земля
Topic 5: пещера, друг, движение, играть, оружие, вме

In [43]:
df[df['cluster']==3]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
8,Tainted Grail: The Fall of Avalon,Добро пожаловать в мир «Tainted Grail: The Fal...,"23 May, 2025","Приключение, Открытый мир, Ролевая игра, Тёмно...",9 / 10,добро пожаловать в мир шагните в мрачную...,добро пожаловать шагните мрачную вселенную рол...,добр пожалова шагн мрачн вселен ролев перв лиц...,добро пожаловать шагнуть мрачный ролевый первы...,3
9,The Elder Scrolls IV: Oblivion Remastered,The Elder Scrolls IV: Oblivion™ Remastered пре...,"22 Apr, 2025","Ролевая игра, Открытый мир, Фэнтези, Для одног...",9 / 10,™ представляет собой переосмысление лучше...,™ представляет собой переосмысление лучшей год...,™ представля соб переосмыслен лучш год потряса...,™ представлять переосмысление хороший год потр...,3
19,Monster Train 2,Власть над Небесами захватили могущественные с...,"21 May, 2025","Стратегия, Карточная игра, Рогалик, Пошаговая ...",10 / 10,власть над небесами захватили могущественные с...,власть небесами захватили могущественные сущес...,власт небес захват могуществен существ тита бы...,власть небо захватить могущественный существо ...,3
42,Blue Prince,Добро пожаловать в таинственный особняк Маунт-...,"10 Apr, 2025","Головоломка, Исследования, Тайна, Расследовани...",9 / 10,добро пожаловать в таинственный особняк маунт ...,добро пожаловать таинственный маунт холли комн...,добр пожалова таинствен маунт холл комнат – ка...,добро пожаловать таинственный маунт холль комн...,3
68,Planet Zoo,Возводите мир для диких животных в игре Planet...,"5 Nov, 2019","Менеджмент, Симулятор, Строительство, Песочниц...",9 / 10,возводите мир для диких животных в игре встр...,возводите диких животных встречайте симулятор ...,возвод дик животн встреча симулятор зоопарк ра...,возводить дикий животное встречать симулятор з...,3
74,THRONE AND LIBERTY,"Добро пожаловать на Throne and Liberty, беспла...","1 Oct, 2024","MMORPG, Бесплатная игра, Открытый мир, ММО, Ро...",6 / 10,добро пожаловать на бесплатную которая про...,добро пожаловать бесплатную которая проходит о...,добр пожалова бесплатн котор проход обширн отк...,добро пожаловать бесплатный проходить обширный...,3
86,Assassin’s Creed Shadows,ИССЛЕДУЙТЕ ФЕОДАЛЬНУЮ ЯПОНИЮИсследуйте завораж...,"19 Mar, 2025","Экшен, Приключение, Открытый мир, Бой, Стелс, ...",7 / 10,исследуйте феодальную япониюисследуйте завораж...,исследуйте феодальную япониюисследуйте завораж...,исслед феодальн япониюисслед заворажива открыт...,феодальный япониюисследовать завораживать откр...,3
92,Stardew Valley,Вам досталась старая дедушкина ферма в долине ...,"26 Feb, 2016","Симулятор фермы, Пиксельная графика, Для неско...",10 / 10,вам досталась старая дедушкина ферма в долине ...,досталась старая дедушкина ферма долине стардь...,доста стар дедушкин ферм долин стард горст мон...,достаться старый дедушкин ферма долина стардие...,3
95,PGA TOUR 2K25,Совершенствуйте точность ударов в PGA TOUR 2K2...,"27 Feb, 2025","Спорт, Гольф, Симулятор, Реализм, Кастомизация...",6 / 10,совершенствуйте точность ударов в новейшей ...,совершенствуйте точность ударов новейшей самог...,совершенств точност удар нов сам популярн голь...,совершенствовать точность удар популярный голь...,3
116,Age of Empires II: Definitive Edition,Отпразднуйте 20-летие одной из самых популярны...,"14 Nov, 2019","Стратегия, Стратегия в реальном времени, Градо...",9 / 10,отпразднуйте летие одной из самых популярных с...,отпразднуйте летие одной самых популярных стра...,отпраздн лет одн сам популярн стратегическ игр...,отпраздновать летие популярный стратегический ...,3


In [165]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=100000000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==3]['text_lemm'])
print('модель LSA:')
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель LSA:
Topic 1: друг, мочь, играть, остров, создать, комната, найти, ждать, вместе, человек
Topic 2: остров, долина, ферма, старый, лошадь, ёкай, вместе, стардие, долина стардие, друг
Topic 3: комната, болванчик, юнит, человек, испытание, номер, одиночка, жуткий, головоломка, мочь играть
Topic 4: долина, ферма, древний, игрушка, забросить, проволока, стардие, долина стардие, зомби, рука
Topic 5: комната, аттракцион, парк, долина, парка, гость, тематический, поле, мочь, верданск
модель NMF:
Topic 1: друг, минь, играть, вечеринка, человек, мочь, сделать, настольный, друг играть, пользователь
Topic 2: остров, долина, ферма, лошадь, открыть, ёкай, друг, мероприятие, секрет, старый
Topic 3: комната, номер, головоломка, мочь играть, играть, маунт холль, маунт, холль, мочь, семинар
Topic 4: стиль, древний, ждать, стать, открытый, друг, помощь, душа, лицо, цивилизация
Topic 5: парка, аттракцион, парк, музей, гость, тематический, дерево, белка, животное, парковый


In [45]:
df[df['cluster']==4]

,title,description,date,tags,rating,prep_text,tokenize_text,text_stem,text_lemm,cluster
3,Cyberpunk 2077,Cyberpunk 2077 — приключенческая ролевая игра ...,"9 Dec, 2020","Киберпанк, Открытый мир, Нагота, Ролевая игра,...",9 / 10,приключенческая ролевая игра с открытым миром...,приключенческая ролевая открытым миром рассказ...,приключенческ ролев открыт мир рассказыва кибе...,приключенческий ролевый открытый рассказывать ...,4
6,Baldur's Gate 3,Соберите отряд и вернитесь в Забытые Королевст...,"3 Aug, 2023","Ролевая игра, Кастомизация персонажа, Решения ...",10 / 10,соберите отряд и вернитесь в забытые королевст...,соберите вернитесь забытые королевства ждет др...,собер верн забыт королевств ждет дружб предате...,собрать вернуться забытый королевство ждать др...,4
7,DOOM: The Dark Ages,СТАНЬТЕ ПАЛАЧОМ В СРЕДНЕВЕКОВОЙ ВОЙНЕ С АДОМDO...,"14 May, 2025","Экшен, Шутер от первого лица, Демоны, Тёмное ф...",9 / 10,станьте палачом в средневековой войне с адомdo...,станьте палачом средневековой войне адомdoom с...,станьт палач средневеков войн адомдо студ зрел...,стать палач средневековый война адомdoom студи...,4
12,Red Dead Redemption 2,"Америка, 1899 год. \r Артур Морган и другие п...","5 Dec, 2019","Открытый мир, Глубокий сюжет, Вестерн, Приключ...",9 / 10,америка год артур морган и другие подручные да...,америка год артур морган другие подручные датч...,америк год артур морга друг подручн датч ван д...,америка год артур морган подручный датч ван де...,4
21,"Warhammer 40,000: Rogue Trader",Преодолевайте невероятные расстояния на своем ...,"7 Dec, 2023","Кастомизация персонажа, Кинематографичная, Глу...",9 / 10,преодолевайте невероятные расстояния на своем ...,преодолевайте невероятные расстояния своем гиг...,преодолева невероятн расстоян сво гигантск кос...,преодолевать расстояние гигантский космический...,4
26,Total War: WARHAMMER III,Предсмертный рев умирающего бога сокрушил гран...,"16 Feb, 2022","Стратегия, Пошаговая стратегия, Глобальная стр...",7 / 10,предсмертный рев умирающего бога сокрушил гран...,предсмертный рев умирающего бога сокрушил гран...,предсмертн рев умира бог сокруш границ мир отк...,предсмертный рёв умирающий бог сокрушить грани...,4
28,Ready or Not,"""Лос-Суэнос. Полиция Лос-Суэноса сообщает о не...","13 Dec, 2023","Тактика, Реализм, Шутер от первого лица, Шутер...",9 / 10,лос суэнос полиция лос суэноса сообщает о неб...,лос суэнос полиция лос суэноса сообщает небыва...,лос суэнос полиц лос суэнос сообща небывал всп...,лос суэнос полиция лос суэнос сообщать небывал...,4
29,Enshrouded,Королевство Эмбервэйл потеряно. В своем алчном...,"24 Jan, 2024","Открытый мир, Выживание, Строительство базы, Д...",9 / 10,королевство эмбервэйл потеряно в своем алчном ...,королевство эмбервэйл потеряно своем алчном ст...,королевств эмбервэйл потеря сво алчн стремлен ...,королевство эмбервэйл потерять алчный стремлен...,4
30,The Witcher 3: Wild Hunt,Одна из самых известных ролевых игр возвращает...,"18 May, 2015","Открытый мир, Ролевая игра, Глубокий сюжет, Ат...",10 / 10,одна из самых известных ролевых игр возвращает...,одна самых известных ролевых игр возвращается ...,одн сам известн ролев игр возвраща нов поколен...,известный ролевый возвращаться поколениявый ге...,4
32,The Last of Us™ Part II Remastered,В этом издании: Одни из нас: Часть II. Обновле...,"3 Apr, 2025","Глубокий сюжет, Постапокалипсис, Шутер от трет...",9 / 10,в этом издании одни из нас часть обновленная ...,издании одни часть обновленная элли эбби одни ...,издан одн част обновлен элл эбб одн част сниск...,часть обновить элли эбби часть снискать наград...,4


In [177]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=100000000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(df[df['cluster']==4]['text_lemm'])
print('модель LSA:')
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")
print('модель NMF:')
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i+1}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

модель LSA:
Topic 1: сила, стать, королевство, жизнь, земля, великий, оружие, мочь, империя, друг
Topic 2: человек паук, паук, контроллер, требоваться, совместимый, майлз, требоваться совместимый, пк, питер, кадр
Topic 3: демон, палач, рок, мрачный, ад, тайна, оружие, кровь, ужас, хаос
Topic 4: демон, великий, легендарный, серия, палач, раса, сетевой, империя, китай, стильный
Topic 5: год, жизнь, семья, боевик, хороший, эпизод, разрешение, увлекательный, часть, вновь
модель NMF:
Topic 1: жизнь, королевство, управлять, семья, играть, строить, выбирать, решение, дом, создать
Topic 2: паук, человек паук, контроллер, требоваться, совместимый, майлз, требоваться совместимый, человек, пк, питер
Topic 3: демон, палач, рок, ад, стильный, палач рок, экшен, боевик, насладиться, возвращаться
Topic 4: великий, империя, век, цивилизация, эпоха, стратегия, исторический, война, сетевой, легендарный
Topic 5: сила, оружие, тайна, пасть, испытание, год, стать, способность, мрачный, лицо


In [47]:
# Восстанавливаем настройки
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

## Классификация

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text_lemm'], df['cluster'], 
                                                      test_size=0.3, 
                                                      random_state=0)

In [50]:
len(X_train)

350

In [51]:
len(X_test)

150

In [52]:
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [53]:
vectorizer = TfidfVectorizer(max_features=10000000, ngram_range=(1,3), stop_words=russian_stopwords)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## LogisticRegression

In [55]:
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)

LogisticRegression()

In [56]:
y_pred = model_lr.predict(X_test_tfidf)

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.08      0.14        13
           1       0.46      1.00      0.63        68
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00        19
           4       1.00      0.05      0.10        39

    accuracy                           0.47       150
   macro avg       0.49      0.23      0.17       150
weighted avg       0.56      0.47      0.32       150



In [58]:
X_test_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

## RandomForestClassifier

In [61]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [62]:
y_pred = model_rf.predict(X_test_tfidf)

In [63]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.08      0.13        13
           1       0.52      0.99      0.68        68
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00        19
           4       0.80      0.41      0.54        39

    accuracy                           0.56       150
   macro avg       0.36      0.29      0.27       150
weighted avg       0.49      0.56      0.46       150



## KNeighborsClassifier

In [65]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_tfidf, y_train)

KNeighborsClassifier()

In [66]:
y_pred = model_knn.predict(X_test_tfidf)

In [67]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.77      0.69        13
           1       0.58      0.81      0.67        68
           2       0.56      0.45      0.50        11
           3       0.33      0.05      0.09        19
           4       0.74      0.51      0.61        39

    accuracy                           0.61       150
   macro avg       0.57      0.52      0.51       150
weighted avg       0.59      0.61      0.57       150



## Сохранение моделей

In [69]:
import pickle

with open('model_knn_games.pkl', 'wb') as f:
    pickle.dump(model_knn, f)

In [70]:
with open('vectorizer_games.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [71]:
df.to_csv('data_games.csv')

In [72]:
import pickle
import joblib
from catboost import CatBoostClassifier
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
with open('model_knn_games.pkl', 'rb') as file:
    model = pickle.load(file)

with open('vectorizer_games.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

In [74]:
def fun_punctuation_text(text):
    text = text.lower()
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    text = ''.join([i if not i.isdigit() else '' for i in text])
    text = ''.join([i if i.isalpha() else ' ' for i in text])
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub('[a-z]', '', text, flags=re.I)
    st = '❯\xa0'
    text = ''.join([ch if ch not in st else ' ' for ch in text])
    return text

In [75]:
def fun_lemmatizing_text(text):
    tokens = word_tokenize(text)
    res = list()
    for word in tokens:
        p = pymorphy3.MorphAnalyzer(lang='ru').parse(word)[0]
        res.append(p.normal_form)  
    text = " ".join(res)
    return text

In [76]:
def fun_tokenize(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    text = " ".join(tokens)
    return text

In [77]:
t1 = "Об этой игре Семичасовая война проиграна. Земля признала поражение. Инцидент в Чёрной Мезе — это лишь далёкое воспоминание. Игрок вновь поднимает монтировку учёного-исследователя Гордона Фримена, обнаружившего себя на наводнённой пришельцами Земле, ресурсы которой стремительно опустошаются, а население сокращается. Фримену навязана незавидная роль спасителя человечества от зла, которое он выпустил на свободу в Чёрной Мезе. И очень многие дорогие ему люди надеются на него. Half-Life 2 — это эпохальный шутер от первого лица, который «заложил основы для следующего поколения игр» (PC Gamer). Вас ждёт захватывающая кампания с беспрецедентной физикой, обеспечивающей невероятный уровень погружения, и головокружительными сражениями. Включает дополнения Episode One и Episode Two Сюжет Half-Life 2 продолжается в двух дополнениях-эпизодах — полноценных приключениях в мире Half-Life, действие которых происходит после основной игры. Они доступны в главном меню, а после прохождения первого дополнения вы сразу же перейдёте ко второму. Half-Life 2: Deathmatch Скоростной многопользовательский экшен во вселенной Half-Life 2! Физика HL2 добавляет битвам насмерть новое измерение. Попробуйте обычный режим «все против всех» или работайте сообща, играя за Альянс или сопротивление. Запустите унитазом в друга уже сегодня! Будет доступно в вашей библиотеке Steam вместе с Half-Life 2. Мастерская Steam Проходите целые кампании и заменяйте оружие, врагов, пользовательский интерфейс и многое другое на контент, созданный сообществом."

In [78]:
t1 = fun_punctuation_text(t1)

In [79]:
t1 = fun_lemmatizing_text(t1)

In [80]:
t1 = fun_tokenize(t1)

In [81]:
text_vectorized = vectorizer.transform([t1])

In [82]:
text_vectorized.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [83]:
prediction = model.predict(text_vectorized)  
probabilities = model.predict_proba(text_vectorized)

In [84]:
print(f"Класс: {prediction[0]}")
print(f"Вероятности: {probabilities}")

Класс: 1
Вероятности: [[0.  0.4 0.4 0.2 0. ]]
